In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_data = pd.read_csv('../train.data',delimiter=',')
train_data = train_data.fillna(0)

In [3]:
matrix = np.zeros((943,1682),dtype=int)
for row in train_data.to_numpy():
    matrix[row[0]-1,row[1]-1] = row[2]
    

In [4]:
Rm = 3
Rmin = 1
Rmax = 5
def Agreement(r1,r2):
    if ((r1>Rm and r2<Rm) or (r1<Rm or r2<Rm)):
        return True
    else :
        return False
def Distance(r1,r2):
    if Agreement(r1,r2):
        return abs(r1-r2)
    else:
        return 2*(abs(r1-r2))
def Proximity(r1,r2):
    return ((2*(Rmax-Rmin)+1)-Distance(r1,r2))**2
def Impact(r1,r2):
    result = (abs(r1-Rm)+1)*(abs(r2-Rm)+1)
    if Agreement(r1,r2):
        return result
    elif not(Agreement(r1,r2)) and result!=0:
        return 1/result
def AvgRating(item):
    sum = matrix[:,item].sum()
    n = np.count_nonzero(matrix[:,item])
    return sum/n
def Popularity(r1,r2,uk):
    if (r1>uk and r2>uk) or (r1<uk or r2<uk):
        return (1+(((r1+r2)/2)-uk)**2)
    else:
        return 1
def PIP(r1,r2,item):
    Rm = 3
    Rmin = 1
    Rmax = 5
    proximity = Proximity(r1,r2)
    impact = Impact(r1,r2)
    item_avg= AvgRating(item)
    popularity = Popularity(r1,r2,item_avg)
    return proximity*impact*popularity
def agPIP(u1,u2,matrix):
    sum=0
    for i in range(1682):
        sum+=PIP(matrix[u1,i],matrix[u2,i],i)
    return sum

In [5]:
pipUser = np.ones((943,943))

In [6]:
for i in range(943):
    for j in range(943):
        if i==j:
            break
        pipUser[i,j] = agPIP(i,j,matrix)
        pipUser[j,i] = pipUser[i,j]

In [7]:
# with open('../pip.csv','w') as f:
#     for i in pipUser:
#         count=0
#         for j in i:
#             f.write(str(j))
#             if count<942:
#                 f.write(',')
#             count+=1
#         f.write('\n')


In [14]:
scaler = MinMaxScaler()
scaler.fit(pipUser)
with open('../pip.csv','w') as f:
        for i in scaler.transform(pipUser):
            count=0
            for j in i:
                f.write(str(j))
                if count<942:
                    f.write(',')
                count+=1
            f.write('\n')